In [1]:
import numpy as np
import cv2
import time 
import os
from tensorflow import keras

In [2]:
modelPath = r'C:\Users\RISHABH\bs final project\Models'
model = keras.models.load_model(modelPath+'/TSModel5')


In [3]:
ImagesFilePath=r'D:\books\4th year 2nd sem books\bs project\dataset\scenes'
ImageNamePath=os.listdir(ImagesFilePath)


In [4]:
def readImage(imagePath):
    img = cv2.imread(ImagesFilePath+'/'+imagePath,1)
    img = cv2.resize(img,(500,400))
    return img

def increaseContrast(img,alpha,beta):
	img=cv2.addWeighted(img,alpha,np.zeros(img.shape,img.dtype),0,beta)
	return img

def returnRedness(img):
	yuv=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
	y,u,v=cv2.split(yuv)
	return v

def threshold(img):
	_,img=cv2.threshold(img,150,255,cv2.THRESH_BINARY)
	return img 

def show(img):
	cv2.imshow('image',img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

def morphology(img,kernelSize=11):
	kernel = np.ones((kernelSize,kernelSize),np.uint8)
	opening = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
	return opening

def findContour(img):
	contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
	return contours

def findBiggestContour(contours):
	m=0
	c=[cv2.contourArea(i) for i in contours]
	return contours[c.index(max(c))]

def boundaryBox(img,contours):
	x,y,w,h=cv2.boundingRect(contours)
	img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
	sign=img[y:(y+h) , x:(x+w)]
	return img,sign

def preprocessingImageToClassifier(image=None,imageSize=28,mu=89.64474891080027,std=53.76704014645958):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    image = cv2.resize(image,(imageSize,imageSize))
    image = (image - mu) / std
    image = image.reshape(1,imageSize,imageSize,1)
    return image


def predict4(sign):
	img=preprocessingImageToClassifier(sign,imageSize=28)
	return np.argmax(model.predict(img))


labelToText={2:"Sign Board",1:"General Board"}

In [ ]:
if __name__ == '__main__':
	for i in ImageNamePath:
		testCase=readImage(i)
		img=np.copy(testCase)
		try:
			img=returnRedness(img)
			img=threshold(img)
			img=morphology(img,11)
			contours=findContour(img)
			big=findBiggestContour(contours)
			testCase,sign=boundaryBox(testCase,big)
			tic=time.time()
			print("Image:",labelToText[predict4(sign)])
			toc=time.time()
		except:
			pass
		show(testCase)
		show(img)